In [2]:
from multiconds import *
import numpy as np

In [3]:
BUTTON_BOX_6

94

Standard main function:

So we want to do all the same stuff, but create a third onset set-- write_PES_conditions, perhaps. That should do it.

And we should probably follow the pattern, and just integrate it with the rest of the code! Since this code already includes `write_betaseries` and `write_conditions`, it would make sense to add a third.

In [3]:
input_dir = "/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/Tasks/SST_DEV/output"

In [4]:
input_dir

'/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/Tasks/SST_DEV/output'

Reconstruct some of the main function just so we can run for a single run, in preparation for creating a function to run all of them.

In [5]:
files = list(Path(input_dir).glob('DEV*.mat'))
files.sort()
pattern = 'DEV(\\d{3})_run(\\d{1})_.*.mat'


In [83]:
print(files[0])

/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/Tasks/SST_DEV/output/DEV004_run1_16-Apr-2018_19-18.mat


In [7]:
f=files[0]

In [8]:
match = re.search(pattern, str(f.name))

In [9]:
if match:
    print('match')

match


In [10]:
subject_id, wave_number = match.groups()
print(f.name)


DEV004_run1_16-Apr-2018_19-18.mat


In [11]:
trial_number, go_no_go_condition, subject_response, reaction_time, trial_duration, trial_start_time = \
    read_data(f)
    
# Create masks for the various conditions
masks = create_masks(go_no_go_condition, subject_response)

In [12]:
print(trial_number, go_no_go_condition, subject_response, reaction_time, trial_duration, trial_start_time)

[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.
  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.
  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.  56.
  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.  70.
  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.  84.
  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.  98.
  99. 100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111. 112.
 113. 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125. 126.
 127. 128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138. 139. 140.
 141. 142. 143. 144. 145. 146. 147. 148. 149. 150. 151. 152. 153. 154.
 155. 156. 157. 158. 159. 160. 161. 162. 163. 164. 165. 166. 167. 168.
 169. 170. 171. 172. 173. 174. 175. 176. 177. 178. 179. 180. 181. 182.
 183. 184. 185. 186. 187. 188. 189. 190. 191. 192. 193. 194. 195. 196.
 197. 

In [13]:
masks
#follows the pattern:     return list((go_success, no_go_success, no_go_fail, null_trials, go_fail))

[array([ True, False, False, False,  True, False, False, False,  True,
        False,  True, False,  True, False,  True, False,  True, False,
         True, False,  True, False,  True, False,  True, False, False,
        False,  True, False, False, False,  True, False, False, False,
         True, False,  True, False,  True, False,  True, False,  True,
        False, False, False, False, False, False, False,  True, False,
         True, False,  True, False,  True, False,  True, False,  True,
        False,  True, False,  True, False, False, False, False, False,
         True, False,  True, False,  True, False,  True, False,  True,
        False,  True, False, False, False, False, False,  True, False,
         True, False,  True, False,  True, False,  True, False,  True,
        False,  True, False, False, False, False, False,  True, False,
         True, False,  True, False,  True, False,  True, False,  True,
        False,  True, False, False, False,  True, False,  True, False,
      

In [14]:
no_go_fail = masks[2]
no_go_success = masks[1]
go = go_no_go_condition==GO_TRIAL

In [15]:
go_no_go_condition==NULL_TRIAL

array([False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True,

In [16]:
len(go_no_go_condition)

256

In [19]:
#marks if each trial is a (successful or failed) go that follows a failed stop
#we shift by 2, not 1, because we ignore the "NULL TRIAL" that occurs reliably every second trial
go_following_failed_stop = np.append([False, False], (go[2:] & no_go_fail[:(len(no_go_fail)-2)]))
#marks if each trial is a (successful or failed) go that follows a successful stop
go_following_successful_stop = np.append([False, False], (go[2:] & no_go_success[:(len(no_go_fail)-2)]))

Now we can write a function that will do this....

In [117]:

def create_pes_masks_from_masks(condition_masks: List) -> List:
    """Create masks of post-error slowing conditions, derived from the original set of masks"""

    go_success = condition_masks[0]
    no_go_success = condition_masks[1]
    no_go_fail = condition_masks[2]
    null_trials = condition_masks[3]
    go_fail = condition_masks[4]

    #go = condition==GO_TRIAL

    #marks if each trial is a (successful or failed) go that follows a failed stop
    #we shift by 2, not 1, because we ignore the "NULL TRIAL" that occurs reliably every second trial
    go_success_following_failed_stop = numpy.append([False, False], (go_success[2:] & no_go_fail[:(len(no_go_fail)-2)]))
    #marks if each trial is a (successful or failed) go that follows a successful stop
    go_success_following_successful_stop = numpy.append([False, False], (go_success[2:] & no_go_success[:(len(no_go_success)-2)]))

    #create one beta for all the other SuccessGo trials
    other_successful_go = go_success & (go_success_following_successful_stop==False) & (go_success_following_failed_stop==False)
    #then just pass on the other masks as returned from create_masks


    #other_failed_go = go_fail & (go_following_successful_stop==False) & (go_following_failed_stop==False)

    #['GoFollowingCorrectStop', 'GoFollowingFailedStop', 
    #'OtherCorrectGo', 'CorrectStop', 'FailedStop', 'Cue', 'OtherFailedGo'
    return list((go_success_following_successful_stop, go_success_following_failed_stop,
                 other_successful_go, no_go_success, no_go_fail, null_trials, go_fail
                ))

In [137]:
def create_conditions(start_time: numpy.ndarray, duration: numpy.ndarray, masks: List,
                      condition_labels: List = ['CorrectGo', 'CorrectStop', 'FailedStop', 'Cue', 'FailedGo']):
    #redesign 2021-10 BJS which only adds conditions with a non-zero amount of trials.
    names = []
    onsets = []
    durations = []
    for i, cond_name in enumerate(condition_labels):
        mask = masks[i]
        if sum(mask)>0:
            names = names + [cond_name]
            cond_onset = start_time[mask].reshape(numpy.count_nonzero(mask), 1)
            onsets = onsets + [cond_onset]
            cond_duration = duration[mask].reshape(numpy.count_nonzero(mask), 1)
            durations = durations + [cond_duration]
#     names = numpy.asarray(['CorrectGo', 'CorrectStop', 'FailedStop', 'Cue', 'FailedGo'], dtype=numpy.object)
#     onsets = numpy.zeros((len(masks),), dtype=numpy.object)
#     durations = numpy.zeros((len(masks),), dtype=numpy.object)
    # onsets and durations have to be reshaped from 1-d numpy arrays to Nx1 arrays so when written
    # by scipy.io.savemat, the correct cell array is created in matlab
    #if a mask has no true values then it's effectively empty.
#     for i, mask in enumerate(masks):
#         onsets[i] = start_time[mask].reshape(numpy.count_nonzero(mask), 1)
#         durations[i] = duration[mask].reshape(numpy.count_nonzero(mask), 1)

    conditions = {'names': numpy.asarray(names,dtype=numpy.object),
                  'onsets': onsets,
                  'durations': durations}
    return conditions

In [138]:
def create_pes_conditions(start_time: numpy.ndarray, duration: numpy.ndarray, pes_masks: List):
    condition_labels =['CorrectGoFollowingCorrectStop', 'CorrectGoFollowingFailedStop',
                           'OtherCorrectGo', 'CorrectStop', 'FailedStop', 'Cue', 'FailedGo']
    return(create_conditions(start_time,duration,pes_masks,condition_labels=condition_labels))



Let's see if we can run these!

In [120]:
pes_masks = create_pes_masks_from_masks(masks)

In [121]:
len(masks)

5

In [122]:
def print_mask_signature(masks):
    print([str(np.sum(m)) + ' True of ' + (str(len(m))) for m in masks])
    print(np.sum([np.sum(m) for m in masks]))

In [123]:
print_mask_signature(masks)

['95 True of 256', '16 True of 256', '16 True of 256', '128 True of 256', '1 True of 256']
256


In [124]:
len(pes_masks)

7

In [125]:
print_mask_signature(pes_masks)

['8 True of 256', '13 True of 256', '74 True of 256', '16 True of 256', '16 True of 256', '128 True of 256', '1 True of 256']
256


Cool, looks good...so can we then create the onset dictionary?

In [140]:
conditions = create_conditions(trial_start_time, trial_duration, masks)
pes_conditions = create_pes_conditions(trial_start_time, trial_duration, pes_masks)

In [127]:
def write_beta_data(input_dir: Union[PathLike, str], subfolder, subject_id: str, wave: str, trials):
    path = Path(input_dir) / subfolder
    path.mkdir(parents=True, exist_ok=True)
    file_name = f'DEV{subject_id}_{wave}_SST1.mat'

    scipy.io.savemat(str(path / file_name), trials)

Now create a new main file with all this integrated.

In [130]:

def main(input_dir: str, bids_dir: str = None, file_limit=None):

    print(input_dir)
	
    files = list(Path(input_dir).glob('DEV*.mat'))
    files.sort()
    pattern = 'DEV(\\d{3})_run(\\d{1})_.*.mat'
    #pattern = 'DEV(\\d{3})_(\\d{1})_SST1\\.mat'
    
    #for testing
    if file_limit is not None:
        files = files[0:file_limit]
    
    for f in files:
        match = re.search(pattern, str(f.name))
        if match:
            subject_id, wave_number = match.groups()
            print(f.name)

            # Read data out of the .mat file
            trial_number, go_no_go_condition, subject_response, reaction_time, trial_duration, trial_start_time = \
                read_data(f)

            # Create masks for the various conditions
            masks = create_masks(go_no_go_condition, subject_response)
            print_mask_signature(masks)
            
            #create masks for the post-error slowing
            pes_masks = create_pes_masks_from_masks(masks)
            print_mask_signature(pes_masks)

            # Perform some quality checking on the numbers of responses (should be 256),
            # the number of null trials (should be 128),
            # go trials (should be 96), and no-go trials (should be 32)
            if subject_response.size != COUNT_RESPONSE:
                print(f'Wrong number of responses   : (subject, expected, actual) '
                      f'({subject_id}, {COUNT_RESPONSE}, {subject_response.size})')
            if numpy.count_nonzero(masks[0] + masks[-1]) != COUNT_GO:
                print(f'Wrong number of go trials : (subject, run, expected, actual) '
                      f'({subject_id}, {wave_number}, {COUNT_GO}, {numpy.count_nonzero(masks[0] + masks[-1])})')
            if numpy.count_nonzero(masks[1] + masks[2]) != COUNT_NO_GO:
                print(f'Wrong number of no-go trials: (subject, expected, actual) '
                      f'({subject_id}, {COUNT_NO_GO}, {numpy.count_nonzero(masks[1] + masks[2])})')
            if numpy.count_nonzero(masks[3]) != COUNT_NULL:
                print(f'Wrong number of null trials : (subject, expected, actual) '
                      f'({subject_id}, {COUNT_NULL}, {numpy.count_nonzero(masks[3])})')

            if bids_dir: #create MAT files storing behavioral information
                trial_type = numpy.empty_like(trial_number, dtype=numpy.object)
                trial_type_names = ['correct-go', 'correct-stop', 'failed-stop', 'null', 'failed-go']
                for mask, name in zip(masks, trial_type_names):
                    numpy.putmask(trial_type, mask, name)
                write_bids_events(bids_dir, subject_id, wave_number,
                                  numpy.stack((trial_start_time, trial_duration, trial_type), axis=1))
            else: #create onset files for SPM first-level analysis
                trials = create_trials(trial_number, trial_start_time, trial_duration)

                # Create paths and file names
                write_betaseries(input_dir, subject_id, wave_number, trials)

                conditions = create_conditions(trial_start_time, trial_duration, masks)
                write_beta_data(input_dir, 'conditions', subject_id, wave_number, conditions)
                
                pes_conditions = create_pes_conditions(trial_start_time, trial_duration, pes_masks)
                write_beta_data(input_dir, 'pes_conditions', subject_id, wave_number, pes_conditions)
            print("written data for subject " + str(subject_id))


In [139]:
main(input_dir,file_limit=10)

/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/Tasks/SST_DEV/output
DEV004_run1_16-Apr-2018_19-18.mat
['95 True of 256', '16 True of 256', '16 True of 256', '128 True of 256', '1 True of 256']
256
['8 True of 256', '13 True of 256', '74 True of 256', '16 True of 256', '16 True of 256', '128 True of 256', '1 True of 256']
256
written data for subject 004
DEV004_run2_15-May-2018_18-41.mat
['94 True of 256', '18 True of 256', '14 True of 256', '128 True of 256', '2 True of 256']
256
['8 True of 256', '10 True of 256', '76 True of 256', '18 True of 256', '14 True of 256', '128 True of 256', '2 True of 256']
256
written data for subject 004
DEV004_run3_03-Sep-2018_12-48.mat
['96 True of 256', '13 True of 256', '19 True of 256', '128 True of 256', '0 True of 256']
256
['8 True of 256', '12 True of 256', '76 True of 256', '13 True of 256', '19 True of 256', '128 True of 256', '0 True of 256']
256
written data for subject 004
DEV004_run4_15-Nov-2018_18-0

The output looks good. So we'll integrate this into the main multiconds.py now.

In [132]:
trial_duration[masks[3]]

array([0.75 , 1.375, 3.125, 1.875, 2.75 , 0.875, 0.5  , 1.   , 0.625,
       1.   , 1.   , 1.5  , 1.   , 0.625, 0.625, 3.5  , 0.75 , 2.75 ,
       0.625, 1.5  , 0.875, 1.125, 0.875, 0.625, 1.625, 0.5  , 0.75 ,
       2.   , 0.5  , 0.625, 0.5  , 0.625, 0.5  , 0.5  , 2.875, 1.125,
       0.625, 1.5  , 0.625, 0.875, 0.75 , 4.   , 1.375, 0.625, 1.25 ,
       3.375, 4.   , 1.25 , 0.75 , 0.625, 1.5  , 1.625, 2.5  , 0.75 ,
       1.25 , 1.125, 1.375, 1.   , 0.5  , 0.625, 0.75 , 0.875, 2.875,
       0.625, 1.5  , 2.   , 2.25 , 0.5  , 1.375, 0.625, 1.75 , 0.5  ,
       0.625, 0.625, 0.5  , 2.   , 0.625, 2.75 , 0.75 , 0.5  , 0.625,
       1.375, 0.625, 1.75 , 0.5  , 1.375, 1.75 , 0.625, 1.   , 1.75 ,
       1.5  , 0.5  , 1.625, 2.125, 2.25 , 0.625, 1.875, 0.875, 0.5  ,
       0.75 , 1.   , 0.5  , 0.5  , 2.5  , 1.75 , 1.25 , 2.625, 1.625,
       0.5  , 0.625, 0.625, 3.   , 2.625, 1.5  , 2.875, 1.   , 0.875,
       1.375, 0.75 , 2.625, 2.5  , 1.5  , 1.125, 0.5  , 0.75 , 1.75 ,
       2.5  , 1.125]

In [133]:
np.sort(np.unique(trial_duration[masks[3]]))

array([0.5  , 0.625, 0.75 , 0.875, 1.   , 1.125, 1.25 , 1.375, 1.5  ,
       1.625, 1.75 , 1.875, 2.   , 2.125, 2.25 , 2.5  , 2.625, 2.75 ,
       2.875, 3.   , 3.125, 3.375, 3.5  , 4.   ])

### Figuring out data available to us

In [88]:
mat_dict = scipy.io.loadmat(str(files[0]), appendmat=False)
response_data = mat_dict['Seeker']

In [90]:
response_data.shape

(256, 17)

    column 0 - trial number
    column 2 - trial type. 0=Go, 1=NoGo, 2=null trial (there are 96 Go, 32 NoGo, 128 null)
    column 6 - subject response (keycode). 0=no response.
    The participant is supposed to respond with left or right hand depending on the stimulus presented.
    Keycodes 91=left, 94=right correspond to (3, 6) buttons on button box in MRI,
    but there are some other keypress pairs that are common:
    (21, 15) correspond to ('r', 'l') keys on keyboard
    (197, 198) correspond to ('<', '>') keys on keyboard
    (92, 93) correspond to (4, 5) buttons on button box in MRI
    (90, 94) correspond to (2, 6) buttons on button box in MRI
    (94, 95) correspond to (6, 7) buttons on button box in MRI (which are on the same hand)
    column 8 - reaction time. For Go trials, this is the duration.
    column 14 - trial duration (seconds).
    column 15 - start time of trial (seconds)

In [96]:
import pandas as pd

In [101]:
response_data_df = pd.DataFrame(response_data)

In [102]:
trial_type = response_data_df[2]

In [103]:
go_trials = trial_type==GO_TRIAL

In [107]:
response_data_df.loc[go_trials,[8,14]].sample(5)

,8,14
0,0.630607,2.25834
230,0.556488,1.94237
4,0.432759,2.25834
14,0.453336,1.94237
152,0.558257,2.51110


Recorded durations and response times are different in the raw.

#### wave 3 sample

In [111]:
run3samplefile=files[2]

PosixPath('/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/Tasks/SST_DEV/output/DEV009_run1_28-Apr-2018_17-21.mat')

In [112]:
mat_dict = scipy.io.loadmat(str(run3samplefile), appendmat=False)
response_data = mat_dict['Seeker']

In [113]:
response_data_df = pd.DataFrame(response_data)
trial_type = response_data_df[2]
go_trials = trial_type==GO_TRIAL
response_data_df.loc[go_trials,[8,14]].sample(5)

,8,14
100,0.475576,2.25834
226,0.513891,1.87917
150,0.503837,1.94237
12,0.480120,2.13195
236,0.437226,2.00556


In [116]:
np.unique(response_data_df.loc[go_trials,14])

array([1.5    , 1.87917, 1.94237, 2.00556, 2.13195, 2.25834, 2.5111 ])

#### handling a file without trials in all conditions

In [142]:
sample_file = '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/Tasks/SST_DEV/output/DEV009_run1_28-Apr-2018_17-21.mat'

In [149]:
f = files[12]
bids_dir=None

In [150]:

match = re.search(pattern, str(f.name))
if match:
    subject_id, wave_number = match.groups()
    print(f.name)

    # Read data out of the .mat file
    trial_number, go_no_go_condition, subject_response, reaction_time, trial_duration, trial_start_time = \
        read_data(f)

    # Create masks for the various conditions
    masks = create_masks(go_no_go_condition, subject_response)
    print_mask_signature(masks)

    #create masks for the post-error slowing
    pes_masks = create_pes_masks_from_masks(masks)
    print_mask_signature(pes_masks)

    # Perform some quality checking on the numbers of responses (should be 256),
    # the number of null trials (should be 128),
    # go trials (should be 96), and no-go trials (should be 32)
    if subject_response.size != COUNT_RESPONSE:
        print(f'Wrong number of responses   : (subject, expected, actual) '
              f'({subject_id}, {COUNT_RESPONSE}, {subject_response.size})')
    if numpy.count_nonzero(masks[0] + masks[-1]) != COUNT_GO:
        print(f'Wrong number of go trials : (subject, run, expected, actual) '
              f'({subject_id}, {wave_number}, {COUNT_GO}, {numpy.count_nonzero(masks[0] + masks[-1])})')
    if numpy.count_nonzero(masks[1] + masks[2]) != COUNT_NO_GO:
        print(f'Wrong number of no-go trials: (subject, expected, actual) '
              f'({subject_id}, {COUNT_NO_GO}, {numpy.count_nonzero(masks[1] + masks[2])})')
    if numpy.count_nonzero(masks[3]) != COUNT_NULL:
        print(f'Wrong number of null trials : (subject, expected, actual) '
              f'({subject_id}, {COUNT_NULL}, {numpy.count_nonzero(masks[3])})')

    if bids_dir: #create MAT files storing behavioral information
        trial_type = numpy.empty_like(trial_number, dtype=numpy.object)
        trial_type_names = ['correct-go', 'correct-stop', 'failed-stop', 'null', 'failed-go']
        for mask, name in zip(masks, trial_type_names):
            numpy.putmask(trial_type, mask, name)
        write_bids_events(bids_dir, subject_id, wave_number,
                          numpy.stack((trial_start_time, trial_duration, trial_type), axis=1))
    else: #create onset files for SPM first-level analysis
        trials = create_trials(trial_number, trial_start_time, trial_duration)

        # Create paths and file names
#        write_betaseries(input_dir, subject_id, wave_number, trials)

        conditions = create_conditions(trial_start_time, trial_duration, masks)
#        write_beta_data(input_dir, 'conditions', subject_id, wave_number, conditions)

        pes_conditions = create_pes_conditions(trial_start_time, trial_duration, pes_masks)
#        write_beta_data(input_dir, 'pes_conditions', subject_id, wave_number, pes_conditions)
    print("written data for subject " + str(subject_id))

DEV009_run1_28-Apr-2018_17-21.mat
['96 True of 256', '0 True of 256', '32 True of 256', '128 True of 256', '0 True of 256']
256
['0 True of 256', '22 True of 256', '74 True of 256', '0 True of 256', '32 True of 256', '128 True of 256', '0 True of 256']
256
written data for subject 009


In [153]:
pes_conditions['names']

array(['CorrectGoFollowingFailedStop', 'OtherCorrectGo', 'FailedStop',
       'Cue'], dtype=object)

In [155]:
len(pes_conditions['onsets'])

4

In [156]:
conditions['names']

array(['CorrectGo', 'FailedStop', 'Cue'], dtype=object)

In [157]:
len(conditions['onsets'])

3